# ACDC Tutorial
This jupyter notebook shows how to run **ASCENT** on ACDC dataset. The dataset used in this tutorial is a cropped version of the original dataset. The original dataset can be found [here](https://humanheart-project.creatis.insa-lyon.fr/database/#collection/637218c173e9f0047faa00fb).

## I. Install dependencies  
The first step is to install the required dependencies. Kindly follow the instructions in the **Install** section in the [README](../README.md) file to create a conda environment and install the required dependencies.
Kindly execute the following cells to install the required dependencies. Make sure you have created a conda environment following the instruction in the README file and activated it. If you have not installed the required dependencies, simply execute the following cell to install them.

In [ ]:
%%capture project_path_setup
import sys

if "../" in sys.path:
    print(sys.path)
else:
    sys.path.append("../")
    print(sys.path)

%%capture packages_install
# Install PyTorch
%conda install pytorch torchvision torchaudio pytorch-cuda=11.8 -c pytorch -c nvidia -y

# Install ASCENt as an editable package
%pip install -e ../.

## II. Dataset and Preprocessing
Once the environment is successfully setup, download the ACDC dataset by executing the following cell. The dataset will be downloaded to the current folder.

In [ ]:
import sys
from pathlib import Path

from tqdm.auto import tqdm

if "./" in sys.path:
    print(sys.path)
else:
    sys.path.append("./")
    print(sys.path)

# Make sure the data is downloaded and extracted where it should be
if not Path("./ACDC").is_dir():
    import zipfile
    from io import BytesIO
    from urllib.request import urlopen

    zipurl = "https://humanheart-project.creatis.insa-lyon.fr/database/api/v1/collection/637218c173e9f0047faa00fb/download"
    with urlopen(zipurl) as zipresp:
        with zipfile.ZipFile(BytesIO(zipresp.read())) as zfile:
            for member in tqdm(
                zfile.infolist(), desc="Downloading and extracting data", position=0, leave=True
            ):
                try:
                    zfile.extract(member, "./")
                except zipfile.error as e:
                    pass

acdc_data_path = "./ACDC/database"

Next, we reformat the dataset to the nnU-Net format. The conversion script we use can be found [here](../ascent/dataset_conversion/acdc.py). We are going to work with a cropped version of ACDC dataset in this tutorial to save time. Execute the following cells to convert the dataset to the nnU-Net format.

In [ ]:
# Executing this cell will show the help message for the ACDC dataset conversion script
%run -i ../ascent/dataset_conversion/acdc.py -h

In [ ]:
# Executing this cell will crop the ACDC dataset and convert it to the nnU-Net format
%run -i ../ascent/dataset_conversion/acdc.py -d {acdc_data_path} -o "../data" -n "ACDC" -cf

# Delete the downloaded data after the conversion
import shutil

shutil.rmtree("./ACDC")

The converted dataset is now located in `../data/ACDC/raw`. We proceed to preprocess the dataset and generate experiment plans by executing the following cell. The preprocessed dataset will be located in `../data/ACDC/preprocessed`. During the preprocessing, the data is cropped to non-zero regions, resampled to median image spacing, and normalized with z-score normalization. The pre-processed data are stored in `.npz` format for faster loading after being unpacked to `.npy`.

> **ⓘ**
> In the next cell, we are using `%run` magic command of jupyter to get a more beautiful and real-time output. In the real case when you are using a terminal, just use `ascent_preprocess_and_plan` command.

In [ ]:
%run ../ascent/preprocess_and_plan.py dataset=ACDC

# Delete the generated log file
import os

os.remove("./preprocess_and_plan.log")

**YAML** plans for training 2D and 3D nnU-Nets are generated and stored in `../ascent/configs`, starting with `acdc_` prefix. Take a look at the generated 3D model plan by executing the following cell.

In [ ]:
from ruamel.yaml import YAML

yaml = YAML(typ="safe")
plan = yaml.load(open("../ascent/configs/model/acdc_3d.yaml"))
print(plan)

## III. Training
We are now ready to train the model. We will train a 3D nnU-Net model on the ACDC dataset using `acdc_3d` experiment plan. We will use tensorboard logger to monitor the training process. Other loggers such as wandb and Comet can also be used. The log folder to save the training logs will be given at the end of the training. Execute the following cell to start the training. We will use reduced number of epochs and batch size to save time and resources. You can keep the default values for better performance.

> **ⓘ**
> By default, the mixed precision is activated to speed up the training process. For Windows users, mixed precision is buggy. You can deactivate it by adding `trainer.precision=32` in the command line arguments in the next cell.  
> **ⓘ**
> In the next cell, we are using `%run` magic command of jupyter to get a more beautiful and real-time output. In the real case when you are using a terminal, just use `ascent_train` command.

In [ ]:
try:
    import ipywidgets
except ModuleNotFoundError:
    %pip install ipywidgets

# Add trainer.accelerator=cpu to train on CPU %run ../ascent/train.py
%run ../ascent/train.py experiment=acdc_2d logger=tensorboard datamodule.test_splits=False datamodule.batch_size=2 fold=0 trainer.max_epochs=2 trainer.precision=32